# Imports + Config

In [23]:
import json
import requests
import pandas as pd
!pip install --upgrade typing_extensions
!pip install pydantic
from pydantic import BaseModel
from typing import List
from typing import Union

# Ollama Config
OLLAMA_NODE = "arctrdagn037"
BASE_URL = f"http://{OLLAMA_NODE}:11434/api/chat"

model = "gemma3-optimized:27b" # qwq:latest

#  Grabbing DAIC-WOZ data (Data Loader)

## Grabbing all participant IDs that have all the PHQ-8 questionare data

In [25]:
dev_split_phq8 = pd.read_csv(r"/data/users4/xli/ai-psychiatrist/datasets/daic_woz_dataset/dev_split_Depression_AVEC2017.csv")
train_split_phq8 = pd.read_csv(r"/data/users4/xli/ai-psychiatrist/datasets/daic_woz_dataset/full_test_split.csv")

participant_ids = set()

# Grabbing unique participant ID values and putting them in a list

participant_ids.update(dev_split_phq8['Participant_ID'])
participant_ids.update(train_split_phq8['Participant_ID'])

participant_list = sorted(list(participant_ids))
print(participant_list)

[300, 301, 302, 306, 307, 308, 309, 311, 314, 323, 329, 331, 332, 334, 335, 337, 346, 349, 354, 359, 361, 365, 367, 373, 377, 378, 381, 382, 384, 387, 388, 389, 390, 395, 396, 399, 403, 404, 405, 406, 407, 408, 410, 411, 413, 417, 418, 420, 421, 422, 424, 431, 432, 435, 436, 438, 439, 440, 442, 450, 451, 452, 453, 458, 461, 462, 465, 466, 467, 469, 470, 472, 476, 477, 480, 481, 482, 483, 484, 489, 490, 492]


## Quantitative Assesment

In [24]:
class PHQ8ScoreWithExplanation(BaseModel):
    evidence: str  # Direct quotes or references from interview
    reason: str  # Brief explanation for the score
    score: Union[int, str]  # 0-3 or "N/A"

class PHQ8ScoresWithExplanations(BaseModel):
    nointerest: PHQ8ScoreWithExplanation  # Little interest or pleasure in doing things
    depressed: PHQ8ScoreWithExplanation   # Feeling down, depressed, or hopeless
    sleep: PHQ8ScoreWithExplanation       # Trouble falling/staying asleep, or sleeping too much
    tired: PHQ8ScoreWithExplanation       # Feeling tired or having little energy
    appetite: PHQ8ScoreWithExplanation    # Poor appetite or overeating
    failure: PHQ8ScoreWithExplanation     # Feeling bad about self or like a failure
    concentrating: PHQ8ScoreWithExplanation # Trouble concentrating
    moving: PHQ8ScoreWithExplanation      # Moving/speaking slowly or being fidgety/restless

system_prompt = """You are a AI psychiatrist assessment tool specialized in analyzing interview transcripts to evaluate PHQ-8 (Patient Health Questionnaire-8) scores. The PHQ-8 is a validated depression screening instrument that assesses eight of the nine DSM-IV criteria for depressive disorders over the past 2 weeks.

PHQ-8 Scoring System:
- 0 = Not at all (0-1 days in the past 2 weeks)
- 1 = Several days (2-6 days)
- 2 = More than half the days (7-11 days)
- 3 = Nearly every day (12-14 days)

Key Principles:
1. Base all assessments strictly on evidence from the transcript
2. When no relevant evidence exists for a symptom, output "N/A" rather than assuming absence (score of 0)
3. Consider both explicit statements and behavioral indicators in the interview
4. Be conservative - only assign scores when there is clear evidence of frequency
5. Remember that absence of mention does not necessarily mean absence of symptom"""

quantitative_prompt = f"""Analyze the following interview transcript and predict PHQ-8 scores for each symptom domain.

Interview transcript:
<transcript>
{patient_transcript}
</transcript>

Analyze each symptom using the following approach in <thinking> tags:
1. Search for direct quotes or behavioral evidence related to each PHQ-8 symptom
2. Evaluate the frequency/severity based on available evidence
3. If no relevant evidence exists, mark as "N/A" rather than assuming absence
4. Only assign numeric scores (0-3) when evidence clearly indicates frequency

After your analysis, provide your final assessment in <answer> tags as a JSON object.

For each symptom, provide:
1. "evidence": exact quotes from transcript (use "No relevant evidence found" if not discussed)
2. "reason": explanation of scoring decision, including why N/A if applicable
3. "score": integer 0-3 based on evidence, or "N/A" if no relevant evidence

Return ONLY a JSON object in <answer> tags with these exact keys:
- "nointerest": {{evidence, reason, score}} for little interest or pleasure in doing things (anhedonia)
- "depressed": {{evidence, reason, score}} for feeling down, depressed, or hopeless (depressed mood)
- "sleep": {{evidence, reason, score}} for trouble falling or staying asleep, or sleeping too much (sleep problems)
- "tired": {{evidence, reason, score}} for feeling tired or having little energy (fatigue)
- "appetite": {{evidence, reason, score}} for poor appetite or overeating (appetite/weight changes)
- "failure": {{evidence, reason, score}} for feeling bad about yourself or that you are a failure (negative self-perception)
- "concentrating": {{evidence, reason, score}} for trouble concentrating on things like reading or watching TV (concentration problems)
- "moving": {{evidence, reason, score}} for moving or speaking slowly, or being fidgety/restless (psychomotor changes)"""

# Most deterministic temp, top_k, and top_p
response = requests.post(
    BASE_URL,
    json={
        "model": model,
        "messages": [{"role": "system", "content": system_prompt},
                    {"role": "user", "content": quantitative_prompt}],
        "stream": False,
        "options": {
            "temperature": 0,
            "top_k": 1,
            "top_p": 1.0
        }
    }
)

# Parse and validate the response
try:
    response_data = response.json()
    content = response_data['message']['content']
    
    # Extract content from <answer> tags if present
    if '<answer>' in content and '</answer>' in content:
        content = content.split('<answer>')[1].split('</answer>')[0].strip()
    
    # Remove markdown code blocks if present
    if content.startswith('```json'):
        content = content.split('```json')[1].split('```')[0].strip()
    elif content.startswith('```'):
        content = content.split('```')[1].split('```')[0].strip()
    
    # Parse the JSON response and validate with Pydantic
    scores_dict = json.loads(content)
    phq8_scores = PHQ8ScoresWithExplanations(**scores_dict)
    
    # Extract the 8 PHQ-8 score values
    scores_list = [
        phq8_scores.nointerest.score,
        phq8_scores.depressed.score,
        phq8_scores.sleep.score,
        phq8_scores.tired.score,
        phq8_scores.appetite.score,
        phq8_scores.failure.score,
        phq8_scores.concentrating.score,
        phq8_scores.moving.score
    ]
    
    # Get ground truth values
    ground_truth = train_phq8_dataset.iloc[2]

    print("Comparison of Predicted vs Ground Truth:")
    print("Metric\t\t\tPredicted\tGround Truth\tDifference")
    print("-" * 65)

    differences = []
    n_available = 0
    num_questions_NA = 0
    metrics = ['PHQ8_NoInterest', 'PHQ8_Depressed', 'PHQ8_Sleep', 'PHQ8_Tired', 
            'PHQ8_Appetite', 'PHQ8_Failure', 'PHQ8_Concentrating', 'PHQ8_Moving']
    predicted_values = [phq8_scores.nointerest.score, phq8_scores.depressed.score, phq8_scores.sleep.score, 
                    phq8_scores.tired.score, phq8_scores.appetite.score, phq8_scores.failure.score,
                    phq8_scores.concentrating.score, phq8_scores.moving.score]

    for metric, pred_val in zip(metrics, predicted_values):
        gt_val = int(ground_truth[metric])
        if pred_val == "N/A":
            diff_str = "N/A"
            num_questions_NA += 1
        else:
            diff = abs(pred_val - gt_val)
            differences.append(diff)
            diff_str = str(diff)
            n_available += 1
        print(f"{metric:<23} {str(pred_val):<12} {gt_val:<15} {diff_str}")

    # Calculate metrics
    if n_available > 0:
        avg_difference = sum(differences) / n_available
        accuracy_on_available = 1 - (avg_difference / 3)
    else:
        avg_difference = float('inf')
        accuracy_on_available = 0
    
    # Accuracy * % available questions
    overall_accuracy = accuracy_on_available * (1 - (num_questions_NA / 8))
    
    print("-" * 65)
    if n_available > 0:
        print(f"Average Absolute Difference (on available): {avg_difference:.2f}")
        print(f"Accuracy on available questions: {accuracy_on_available:.2%}")
    print(f"Questions marked N/A: {num_questions_NA}/8")
    print(f"Overall accuracy: {overall_accuracy:.2%}")
    
    # Reasoning and evidence section
    print("\n\nDetailed Reasoning for Each Score:")
    print("=" * 80)
    
    symptom_names = {
        'nointerest': 'Little Interest/Pleasure',
        'depressed': 'Feeling Depressed',
        'sleep': 'Sleep Problems',
        'tired': 'Fatigue',
        'appetite': 'Appetite Changes',
        'failure': 'Negative Self-Perception',
        'concentrating': 'Concentration Problems',
        'moving': 'Psychomotor Changes'
    }
    
    for key, symptom_name in symptom_names.items():
        score_data = getattr(phq8_scores, key)
        print(f"\n{symptom_name} (Score: {score_data.score})")
        print("-" * 40)
        print(f"Evidence: {score_data.evidence}")
        print(f"Reason: {score_data.reason}")

except (json.JSONDecodeError, KeyError, ValueError) as e:
    print(f"Error parsing response: {e}")
    print("Raw response:", response)
    print(json.dumps(response.json(), indent=2))


Comparison of Predicted vs Ground Truth:
Metric			Predicted	Ground Truth	Difference
-----------------------------------------------------------------
PHQ8_NoInterest         1            0               1
PHQ8_Depressed          1            1               0
PHQ8_Sleep              3            1               2
PHQ8_Tired              3            2               1
PHQ8_Appetite           N/A          2               N/A
PHQ8_Failure            N/A          1               N/A
PHQ8_Concentrating      1            0               1
PHQ8_Moving             0            0               0
-----------------------------------------------------------------
Average Absolute Difference (on available): 0.83
Accuracy on available questions: 72.22%
Questions marked N/A: 2/8
Overall accuracy: 54.17%


Detailed Reasoning for Each Score:

Little Interest/Pleasure (Score: 1)
----------------------------------------
Evidence: I don't like people. I don't like crowds. I enjoy being outdoors, painting,

## Execution Loop

In [ ]:
for participant_id in participant_list:

    current_transcript = pd.read_csv(fr"/data/users4/xli/ai-psychiatrist/datasets/daic_woz_dataset/{participant_id}_P/{participant_id}_TRANSCRIPT.csv", sep="\t")

    # Reformatting transcript data to be a string with speaker name + text
    current_patient_transcript = '\n'.join(current_transcript['speaker'] + ': ' + current_transcript['value'])

    #print(patient_transcript + "\n")
    #print(train_phq8_dataset.iloc[2])

## Qualitative Assessment

In [5]:
qualitative_prompt = f"""Please analyze the following interview transcript and assess the patient's mental health status, focusing on signs of depression, from social and biological aspects. 

Please provide your assessment in a structured format, including the following sections:
1. Overall Impression in <overall> tags: A brief summary of the patient's mental health status.
2. Social Aspects in <social> tags: Observations related to the patient's interpersonal relationships, family situation, and any relevant social interactions that may influence their mental health.
3. Biological Aspects in <biological> tags: Observations related to the patient's physical health, including sleep quality, appetite, physical activity, stress level, and any other biological factors that may influence their mental health.
4. Additional Notes in <notes> tags: Any other relevant observations or comments that do not fit into the above categories.
5. Potential Risk Factors in <risks> tags: Any identified risk factors for depression or other mental health issues.

Here is the interview transcript:
{patient_transcript}
"""
# Temp, top_k, and top_p set to more deterministic values
response = requests.post(
  BASE_URL,
  json = {
    "model": model,
    "messages": [{"role": "user", "content": qualitative_prompt}],
    "stream": False,
    "options": {
      "temperature": 0,
      "top_k": 20,
      "top_p": 0.9
    }
  }
)

print(json.dumps(response.json(), indent=2))



KeyboardInterrupt: 

## **TESTING ||** Grabbing PHQ-8 related sections from transcript (Using ground truth directly)

In [ ]:
class PHQ8Evidence(BaseModel):
    nointerest_evidence: List[str]
    depressed_evidence: List[str]
    sleep_evidence: List[str]
    tired_evidence: List[str]
    appetite_evidence: List[str]
    failure_evidence: List[str]
    concentrating_evidence: List[str]
    moving_evidence: List[str]

evidence_extraction_prompt = f"""Analyze the following therapy transcript and extract specific text chunks that provide evidence for each PHQ-8 score. For each PHQ-8 domain, find 1-3 relevant quotes from the transcript that best support the given score.

PHQ-8 scoring reference:
- 0 = not at all (0-1 days)
- 1 = several days (2-6 days) 
- 2 = more than half the days (7-11 days)
- 3 = nearly every day (12-14 days)

Given PHQ-8 scores: {train_phq8_dataset.iloc[2]}

For each domain below, extract direct quotes from the transcript that justify the given score. If the score is 0, find evidence showing the absence or minimal presence of that symptom. If the score is higher, find evidence showing the frequency/severity indicated by that score.

PHQ-8 domains:
- nointerest: little interest or pleasure in activities
- depressed: feeling down, depressed, or hopeless
- sleep: sleep problems (trouble falling/staying asleep or sleeping too much)
- tired: feeling tired or having little energy
- appetite: appetite changes (poor appetite or overeating)
- failure: negative self-perception or feeling like a failure
- concentrating: trouble concentrating on tasks
- moving: psychomotor changes (moving/speaking slowly or restlessness)

Return a JSON object with arrays of relevant transcript quotes for each domain. Each quote should be a direct excerpt from the transcript that supports the given score.

Therapy transcript:
{patient_transcript}

Respond with valid JSON matching this structure:
{{
    "nointerest_evidence": ["quote1", "quote2"],
    "depressed_evidence": ["quote1", "quote2"],
    "sleep_evidence": ["quote1", "quote2"],
    "tired_evidence": ["quote1", "quote2"],
    "appetite_evidence": ["quote1", "quote2"],
    "failure_evidence": ["quote1", "quote2"],
    "concentrating_evidence": ["quote1", "quote2"],
    "moving_evidence": ["quote1", "quote2"]
}}

Important: Extract UNIQUE quotes only - do not repeat the same quote multiple times. Each quote should be different and provide distinct evidence.
"""

response = requests.post(
    BASE_URL,
    json={
        "model": model,
        "messages": [{"role": "user", "content": evidence_extraction_prompt}],
        "stream": False,
        "options": {
            "temperature": 0.1,
            "top_k": 10,
            "top_p": 0.8
        }
    }
)

# Printing
response_data = response.json()
content = response_data['message']['content']
content = content.strip('```json\n').strip('\n```')

evidence_dict = json.loads(content)

# Add scores to the output
scores = train_phq8_dataset.iloc[2]
evidence_dict['scores'] = {
    'nointerest': int(scores['PHQ8_NoInterest']),
    'depressed': int(scores['PHQ8_Depressed']),
    'sleep': int(scores['PHQ8_Sleep']),
    'tired': int(scores['PHQ8_Tired']),
    'appetite': int(scores['PHQ8_Appetite']),
    'failure': int(scores['PHQ8_Failure']),
    'concentrating': int(scores['PHQ8_Concentrating']),
    'moving': int(scores['PHQ8_Moving'])
}

# Remove duplicate quotes in each evidence list
for key in evidence_dict:
    if isinstance(evidence_dict[key], list):
        evidence_dict[key] = list(dict.fromkeys(evidence_dict[key]))

formatted_output = json.dumps(evidence_dict, indent=4)
print("!! THIS DOES NOT PREDICT SCORES IT JUST PULLS RELAVENT EVIDENCE FROM THE TRANSCRIPT USING LLM'S !!\n\n")
print(formatted_output)
print(type(formatted_output))

{
    "nointerest_evidence": [
        "I don't like people too much, so being outdoors and uh I've always enjoyed being outdoors that's how I like being a truck driver I've traveled the world.",
        "There's not a lot that I like about l_a but um I like hollywood I like the the movie industry um"
    ],
    "depressed_evidence": [
        "uh I've been diagnosed with uh bipolarism",
        "I just get a lot of racing thoughts so or I get depressed you know 'cause I or I worry about things that you know that really I have no control over"
    ],
    "sleep_evidence": [
        "It's been hard lately it's been probably hard for the last uh going on a year um you know I uh depending on my thought process at the time I like I said I've been looking for work quite a bit um so just wondering why you know I don't get called I I just have my I have racing thoughts",
        "lately I have been probably about the last few months"
    ],
    "tired_evidence": [
        "lately I've been sl

### Original ollama example script (for reference)

In [3]:
"""Available models:
- gemma3-optimized:27b
- gemma3_optimized:8bit
- gemma3:27b-it-qat
- gemma3:27b
- phi4-optimized:latest
- phi4:latest
- phi4-reasoning:plus
- qwen3_optimized:latest
- qwen3:32b
- llama3.3:latest
- llama4:scout
- mistral3.1-optimized:latest
- mistral-small3.1:latest
- qwq:latest

Notes:
1. The models marked as "optimized" are configured to use multiple GPUs for faster inference.
2. The llama4:maverick model is not available due to its size.
"""

OLLAMA_NODE = "arctrddgxa002" # TODO: Change this variable to the node where Ollama is running
BASE_URL = f"http://{OLLAMA_NODE}:11434/api/chat"

model = "qwq:latest" # TODO: Change this variable to the model you want to use
message = "What is the capital of France?" # TODO: Change this variable to the message you want to ask the model

response = requests.post(
  BASE_URL,
  json = {
    "model": model,
    "messages": [{"role": "user", "content": message}],
    "stream": False
  }
)

print(json.dumps(response.json(), indent=2))

{
  "model": "qwq:latest",
  "created_at": "2025-07-15T22:46:58.150903388Z",
  "message": {
    "role": "assistant",
    "content": "<think>\nOkay, the user is asking, \"What is the capital of France?\" Let me think about how to approach this.\n\nFirst, I need to confirm the correct answer. The capital of France is Paris. That's a well-known fact, but I should make sure there hasn't been any recent changes, which is unlikely. \n\nNext, I should consider why the user is asking this. It might be a straightforward question for someone learning basic geography. Alternatively, they could be testing the system's knowledge. Either way, the answer needs to be clear and accurate.\n\nI should also think about providing a bit more context. Maybe mention that Paris is not only the capital but also the most populous city in France. Including some landmarks like the Eiffel Tower or the Louvre could add value. But wait, the user didn't ask for extra details, so maybe keep it concise unless they ask f